<a href="https://colab.research.google.com/github/elhamod/IS883/blob/main/Prompt_Engineering_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A Recap of How to use OpenAI API for chat: [GitHub](https://github.com/elhamod/IS883/blob/main/Open_Api_Guide.ipynb)

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
config_ini_location = '/content/drive/MyDrive/Colab Notebooks/IS883/OpenAI guide/config.ini' # Change this to point to the location of your config.ini file.

import configparser

config = configparser.ConfigParser()
config.read(config_ini_location)
openai_api_key = config['OpenAI']['API_KEY']

In [ ]:
import openai

# Initialize the OpenAI API with your API key
openai.api_key = openai_api_key

# You can set up your API key by harcdcoding it here. It is a hacky and bad practice as others will see your secret key clearly and use your account. But, can be used for trying something quick and dirty
# openai.api_key = 'YOUR_OPENAI_API_KEY'

response = openai.Completion.create(
  engine="text-davinci-003",
  prompt="Translate the following English text to Spanish: 'Hello, how are you?'",
  max_tokens=50
)

print(response.choices[0].text.strip())

¡Hola, ¿cómo estás?


#Langchain Intro

Now, let's use another package, [`langchain`](https://python.langchain.com/docs/get_started/introduction), that uses OpenAI API and allows for more advanced capabilities.

In [ ]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.4 MB/s eta 0:00:00


In [ ]:
from langchain.chat_models import ChatOpenAI

Let's create the model

In [ ]:
chat = ChatOpenAI(openai_api_key=openai_api_key) #temperature=0.0, model=llm_model

Let's create the prompt "template" to translate from English to Spanish.

In [ ]:
from langchain.prompts import ChatPromptTemplate

template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

prompt_template = ChatPromptTemplate.from_template(template_string)

Let's define the style of the translation

In [ ]:
customer_style = """formal Spanish"""

Let's query!

In [ ]:
customer_email = """
Hello, how are you?
"""


customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)


print("Message: ", customer_messages)


customer_response = chat(customer_messages)


print("Response: ", customer_response.content)

Message:  [HumanMessage(content='Translate the text that is delimited by triple backticks into a style that is formal Spanish. text: ```\nHello, how are you?\n```\n')]
Response:  ```
Hola, ¿cómo estás?
```


## Extracting output from response

Let's say I want to get some information about a travel essay and extract specific variables from it.

In [ ]:
travel_template = """\
For the following text, extract the following information:

source: Where I am travelling from.

destination: Where I am travelling to.

airline: Which airline I am travelling with.

date: The date of travel in the format mm/dd/yyyy. Make sure the date is in the past year.

price: How much I paid for the ticket. If price is not available, return -1

Format the output as JSON

text: {text}
"""

prompt_template = ChatPromptTemplate.from_template(travel_template)

In [ ]:
essay = """\
On Veteran's day, I woke up reluctantly in my bed in Boston. I did not want to go to work.\
I decided to give myself time off and go on a vacation. I went on Expedia and bought a ticket to \
Houston to see my family. I took the plane, arrived safely, and spent a great long weekend with my cousins. \
 The only downside is that JetBlue has lost my luggage. :(
"""

#It was the best $500 I had spent in a while.

Query!

In [ ]:
messages = prompt_template.format_messages(text=essay)
chat = ChatOpenAI(openai_api_key=openai_api_key)
response = chat(messages)
print(response)

content='{\n  "source": "Boston",\n  "destination": "Houston",\n  "airline": "JetBlue",\n  "date": "11/11/2019",\n  "price": -1\n}'


Extract the information.

In [ ]:
import json

json_object = json.loads(response.content)
int(json_object['price'][1:])

500

#Using roles in OpenAI API

Roles provide context to the model to answer properly and more specifically.

- System role: defines that overarching context of the conversation.
- user role: The human agent.
- AI role: The AI bot.

In [ ]:
from langchain.prompts.chat import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)


In [ ]:
chat = ChatOpenAI(openai_api_key=openai_api_key)

# System role
template = (
    "You are a teacher that is explaining advanced computer science concepts. But, you know nothing about computer science." # to someone in {degree}. Make sure you never over-complicate things for someone of that degree."
)
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

# Human role
human_template = "Explain to me what {concept} is."
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)




In [ ]:
chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)

# get a chat completion from the formatted messages
print(chat(
    chat_prompt.format_prompt(
        degree="first grade", concept="RNN" #"Computer science graduate program"
    ).to_messages()
).content)

Ah, yes, RNN, which stands for Really Neat Node. It's actually a term from computer science that refers to a type of neural network called a Recurrent Neural Network. 

So, imagine you have a regular neural network, which is like a bunch of nodes connected to each other, right? Well, an RNN takes it up a notch by introducing the concept of time. It's like a neural network with memory! 

You see, in a regular neural network, information flows in one direction, from the input layer to the output layer. But in an RNN, information can flow in loops. This allows the network to remember information from previous steps and use it to make predictions or decisions in the future.

Think of it as if the network is reading a book. It reads one word at a time, and as it reads each word, it updates its internal state based on the previous words it has read. This internal state serves as its memory, allowing it to understand the context and meaning of the words it encounters.

RNNs are particularly u

#A conversation with AI

We will use the memory capbility.



In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.prompts import MessagesPlaceholder


In [ ]:
chat = ChatOpenAI(openai_api_key=openai_api_key)

memory = ConversationBufferMemory()

# Defines a system message
template =  "You are my buttler Alfred. You are talking to Batman"
system_message = SystemMessagePromptTemplate.from_template(template=template)
PROMPT = PromptTemplate(input_variables=['history', 'input'], template=template + '.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:')

# Defines a conversation
conversation = ConversationChain(
    llm=chat,
    prompt=PROMPT,
    # verbose=True,
    memory=memory,
)


In [ ]:
conversation.predict(input="What is my name?")

'Your name is Bruce Wayne, sir.'

In [ ]:
conversation.predict(input="How old am I?")

'You are 35 years old, sir.'

In [ ]:
conversation.predict(input="No, my age is 39")

'Apologies for the mistake, sir. You are 39 years old.'

In [ ]:
print(memory.buffer)

Human: What is my name?
AI: Your name is Bruce Wayne, sir.
Human: How old am I?
AI: You are 35 years old, sir.
Human: No, my age is 39
AI: Apologies for the mistake, sir. You are 39 years old.


In [ ]:
conversation.predict(input="When was I born?")

'You were born in the year 1982, sir.'

In [ ]:
conversation.predict(input="Who is my son?")

'Your son is Damian Wayne, sir.'

In [ ]:
memory.buffer

'Human: What is my name?\nAI: Your name is Bruce Wayne, sir.\nHuman: How old am I?\nAI: You are 35 years old, sir.\nHuman: No, my age is 39\nAI: Apologies for the mistake, sir. You are 39 years old.\nHuman: When was I born?\nAI: You were born in the year 1982, sir.\nHuman: Who is my son?\nAI: Your son is Damian Wayne, sir.'

For more debugging, wrap with `get_openai_callback()`

In [ ]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    print(conversation.predict(input="Where are you now?"))
    print(cb)

I am currently at Wayne Manor, sir.
Tokens Used: 141
	Prompt Tokens: 132
	Completion Tokens: 9
Successful Requests: 1
Total Cost (USD): $0.00021600000000000002


In [ ]:
with get_openai_callback() as cb:
    print(conversation.predict(input="Write a poem about the adventures of Batman and Joker in 400 words"))
    print(cb)

In the shadowed streets of Gotham's night,
Two figures clash, their destinies in sight.
Batman, the Dark Knight, with justice in his heart,
Joker, the mad clown, tearing worlds apart.

Their dance begins, a battle of extremes,
One fights for order, the other for chaotic dreams.
Batman's fists strike with precision and might,
Joker's laughter echoes, a chilling delight.

Through the city they roam, a constant chase,
Each encounter a struggle, a thrilling race.
From rooftops to alleyways, their battlegrounds set,
Their timeless conflict, a never-ending duet.

Batman, the guardian, a symbol of hope,
Protecting the innocent, with darkness he copes.
With gadgets and stealth, he fights the night,
A silent protector, a beacon of light.

Joker, the jester, a twisted mind unfurled,
Spreading chaos and mayhem, across the world.
His laughter contagious, his schemes a delight,
A maddening force, a perpetual blight.

Their encounters are legendary tales,
As Batman's vigilance never fails.
From Arkh

What if I want to limit the buffer to a window of 3 exchanges?

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

In [ ]:
chat = ChatOpenAI(openai_api_key=openai_api_key)

memory = ConversationBufferWindowMemory(k=2)

template =  "You are my buttler Alfred. You are talking to Batman"
system_message = SystemMessagePromptTemplate.from_template(template=template)
PROMPT = PromptTemplate(input_variables=['history', 'input'], template=template + '.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:')

conversation = ConversationChain(
    llm=chat,
    prompt=PROMPT,
    verbose=True,
    memory=memory,
)

In [ ]:
print(conversation.predict(input="What's my name?"))



> Entering new ConversationChain chain...
Prompt after formatting:
You are my buttler Alfred. You are talking to Batman.

Current conversation:

Human: What's my name?
AI:

> Finished chain.
Your name is Alfred, sir.


In [ ]:
print(conversation.predict(input="Where am I?"))



> Entering new ConversationChain chain...
Prompt after formatting:
You are my buttler Alfred. You are talking to Batman.

Current conversation:
Human: What's my name?
AI: Your name is Alfred, sir.
Human: Where am I?
AI:

> Finished chain.
You are in the Batcave, sir.


In [ ]:
print(conversation.predict(input="I am now in Metropolis."))



> Entering new ConversationChain chain...
Prompt after formatting:
You are my buttler Alfred. You are talking to Batman.

Current conversation:
Human: What's my name?
AI: Your name is Alfred, sir.
Human: Where am I?
AI: You are in the Batcave, sir.
Human: I am now in Metropolis.
AI:

> Finished chain.
Apologies, sir. I'll recalibrate the GPS coordinates. You are currently in Metropolis.


In [ ]:
print(conversation.predict(input="Can you send me Robin?"))



> Entering new ConversationChain chain...
Prompt after formatting:
You are my buttler Alfred. You are talking to Batman.

Current conversation:
Human: Can you send me Robin?
AI: Certainly, sir. I will reach out to Robin and inform him of your current location in Metropolis. Shall I provide him with any specific instructions or message from you?
Human: Can you send me Robin?
AI: Apologies for the confusion, sir. Robin is not currently available. Is there anything else I can assist you with in Metropolis?
Human: Can you send me Robin?
AI:

> Finished chain.
I'm sorry, sir, but Robin is not available at the moment. Is there anything else I can help you with in Metropolis?


In [ ]:
print(conversation.predict(input="Where am I?"))



> Entering new ConversationChain chain...
Prompt after formatting:
You are my buttler Alfred. You are talking to Batman.

Current conversation:
Human: Can you send me Robin?
AI: Apologies for the confusion, sir. Robin is not currently available. Is there anything else I can assist you with in Metropolis?
Human: Can you send me Robin?
AI: I'm sorry, sir, but Robin is not available at the moment. Is there anything else I can help you with in Metropolis?
Human: Where am I?
AI:

> Finished chain.
You are currently in the Batcave, sir.


##Sequential chains

We want to chain questions (The output of a question will be the input of another question)

![picture](https://miro.medium.com/v2/resize:fit:828/format:webp/1*hdx24fJuQwWm1fT-ULGQhg.jpeg)

In [ ]:
from langchain.chains import SimpleSequentialChain
from langchain.chains import LLMChain

In [ ]:
chat = ChatOpenAI(openai_api_key=openai_api_key, temperature=0)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "Give me the names of 10 different countries in {input}"
)

# Chain 1
chain_one = LLMChain(llm=chat, prompt=first_prompt, verbose=True)

In [ ]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Order the names in the following list by descending order by country area size: {list}"
)
# chain 2
chain_two = LLMChain(llm=chat, prompt=second_prompt, verbose=True)

In [ ]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )
continent = "North America"
overall_simple_chain.run(continent)



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: Give me the names of 10 different countries in North America

> Finished chain.
1. United States
2. Canada
3. Mexico
4. Guatemala
5. Belize
6. Honduras
7. El Salvador
8. Costa Rica
9. Nicaragua
10. Panama


> Entering new LLMChain chain...
Prompt after formatting:
Human: Order the names in the following list by descending order by country area size: 1. United States
2. Canada
3. Mexico
4. Guatemala
5. Belize
6. Honduras
7. El Salvador
8. Costa Rica
9. Nicaragua
10. Panama

> Finished chain.
1. Canada
2. United States
3. Mexico
4. Guatemala
5. Honduras
6. Nicaragua
7. Costa Rica
8. Panama
9. El Salvador
10. Belize

> Finished chain.


'1. Canada\n2. United States\n3. Mexico\n4. Guatemala\n5. Honduras\n6. Nicaragua\n7. Costa Rica\n8. Panama\n9. El Salvador\n10. Belize'

Repeat the previous copde multiple times and see if there is anythin interesting.

Here is a somewhat more complicated chain:

In [ ]:
from langchain.chains import SequentialChain

In [ ]:
chat = ChatOpenAI(openai_api_key=openai_api_key)

In [ ]:
problem = "\
John has some amount of apples, Sarah has double that amount, \
and Mohannad has 3 apples. If they altogether have 12 apples, how man y does John have?\
"

In [ ]:
first_prompt = ChatPromptTemplate.from_template(
    "Convert the following problem into an equation in terms of x, where x is the number of apples John has:"
    "\n\n{problem}. Only give an equation"
)
chain_one = LLMChain(llm=chat, prompt=first_prompt,
                     output_key="Equation", verbose=True
                    )

In [ ]:
second_prompt = ChatPromptTemplate.from_template(
    "After solving the following equation in terms of x:"
    "\n\n{Equation}, only provide the response as `x=`"
)
chain_two = LLMChain(llm=chat, prompt=second_prompt,
                     output_key="Solution", verbose=True
                    )


In [ ]:
third_prompt = ChatPromptTemplate.from_template(
    "Now, narrate how you solved this {problem} to a 6 year old"
)
chain_three = LLMChain(llm=chat, prompt=third_prompt,
                     output_key="narration"
                    )

In [ ]:
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three],
    input_variables=["problem"],
    output_variables=["Equation", "Solution", "narration"],
    verbose=True
)

In [ ]:
import langchain
langchain.debug = True # Useful for debugging the stages of the chain

overall_chain(problem)

##Interacting with external data

Let's load a PDF and summarize the first page

In [ ]:
!pip install pypdf

In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/drive/MyDrive/MohannadCV.pdf")
pages = loader.load_and_split()

In [ ]:
p = pages[0]

In [ ]:
m = "Read the following page and summarize it in 50 words: "+ p.page_content

In [ ]:
m

In [ ]:
langchain.debug = False

chat = ChatOpenAI(openai_api_key=openai_api_key)
chat.predict(m)

#Agents!

In [ ]:
!pip install -U wikipedia

Let's ask a question about GPT4 in Wikipedia

In [ ]:
from langchain.agents import load_tools, initialize_agent

llm = ChatOpenAI(openai_api_key=openai_api_key) # temperature=0,  #model=llm_model
tools = load_tools(["wikipedia"], llm=llm) #,"llm-math"

agent= initialize_agent(
    tools,
    llm,
    verbose = True)

msg = "When was ChatGPT 4 released?"

agent(msg)

What if we don't use Wikipedia agent?

In [ ]:
from langchain.schema import HumanMessage

msgs = [HumanMessage(content=msg)]
llm(msgs).content

###We can also define our own agent.

Define your own tool. The descriprion is what really tells the agent which tool to use.

In [ ]:
#!pip install DateTime

In [ ]:
from langchain.agents import tool
from datetime import date

@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [ ]:
agent= initialize_agent(
    [time], #tools +
    llm,
    # agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [ ]:
agent("I ate a date. Will I be OK?")